In [1]:
import pandas as pd
import json, os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8" #CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8

import torch
import torch.nn as nn
import torch_geometric
from torch_geometric.nn import GCNConv, GatedGraphConv, GATConv, SuperGATConv, MLP, GraphSAGE
import torch.nn.functional as F
from torch.nn import Linear, LSTM, RNN, GRU, ReLU, Tanh, Sigmoid, CrossEntropyLoss

from torch.optim import Adam, SGD
from torch.nn.functional import cross_entropy
from torch.utils.data import random_split

from torch_geometric.data import Data, Batch
from torch_geometric.explain import Explainer, GNNExplainer, CaptumExplainer, PGExplainer, AttentionExplainer
from torch_geometric.nn import global_mean_pool, BatchNorm, global_max_pool, global_add_pool, TopKPooling, SAGPooling

from captum.attr import Saliency, IntegratedGradients

import numpy as np
import sklearn
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import KFold, StratifiedKFold

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
print('Device:', device)

if device == 'cuda':
    #torch.use_deterministic_algorithms(True)
    torch.backends.cudnn.benchmark = False
#     os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8" #CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8

torch.manual_seed(11)
torch_geometric.seed_everything(11)
np.random.seed(11)

Device: cuda


In [2]:
### Define GNN model ###
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, int(hidden_channels/2))
        self.linear = Linear(int(hidden_channels/2), int(hidden_channels/4))
        self.lstm = LSTM(int(hidden_channels/4), num_classes)
    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings: Embed each node by performing multiple rounds of message passing
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.linear(x)
        # 2. Readout layer: Aggregate node embeddings into a unified graph embedding (readout layer)
        x = global_mean_pool(x, batch) #, TopKPooling, SAGPooling# global_mean_pool(x, batch)
        # 3. Apply a final classifier: Train a final classifier on the graph embedding
        x = F.dropout(x, p=0.5, training=self.training)
        return F.log_softmax(x, dim=1)

def train_model(num_features, hidden_channels, epochs, train_batch):
    # Create an instance of the model
    model = GCN(num_features, hidden_channels, 2).to(device)
    # Define the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()

    # Train Model
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(train_batch.x, train_batch.edge_index, train_batch.batch)
        loss = criterion(out, train_batch.y)
        loss.backward()
        optimizer.step()
    return model

def eval_model(model, test_batch):
    # Evaluate Model
    model.eval()
    out = model(test_batch.x, test_batch.edge_index, test_batch.batch)
    pred = out.argmax(dim=1).tolist()  # Use the class with highest probability.
    return pred

In [18]:
# Read the data from the csv file
df = pd.read_csv('Dynamic_Dataset_v8_without_missing_values.csv', sep=',')

x_columns = ['isEntre', # 'Segment_id' 'isQ&A_v2', 'Duration (secs)', 'Ethnicity==White', 'Asked_Amount_USD','Asked_Equity_PCT', 
    'isQ&A_v2', 'Duration (secs)',
    'Age', 'Ethnicity==Black', 'Industry==Children / Education', 'Industry==Fashion / Beauty', 'Industry==Fitness / Sports / Outdoors', 
    'Industry==Food and Beverage', 'Industry==Health / Wellness / Cleaning', 'Industry==Lifestyle / Home', 'Industry==Software / Tech', 'Industry==Pet Products', 
    'Revenue_Model==Production/Transactional model', 'Revenue_Model==Subscription model', 'Revenue_Model==Rental or leasing model', 'Retail_Ecommerce==Retail', 
    'Retail_Ecommerce==Online', 'Has_Patent==YES', 'Has_Patent==IN-PROGRESS', 'Num_of_Presenters', 'Has_Debt', 'Seasonal', 'Num_of_Sales_Last_Year_USD_resc', 
    'VOICE_pitch', 'VOICE_articulation_rate', 'VOICE_neutral', 'VOICE_calm', 'VOICE_happy', 'VOICE_sad', 'VOICE_angry', 
    'FACE_smiling', 'FACE_happy', 'FACE_neutral', 'FACE_sad', 'FACE_angry', 
    'TEXT_financial_sentiment_score', 'TEXT_generic_sentiment_score', 'TEXT_joy', 'TEXT_sadness', 'TEXT_anger', 
    'TEXT_trust', 'TEXT_conflict', 'TEXT_social_support', 'TEXT_similarity', 'TEXT_respect', 'TEXT_knowledge', 'TEXT_power', 'TEXT_fun',  'TEXT_identity', 'TEXT_romance',
    'TEXT_lexical_diversity_mtld', 'TEXT_lexical_sophistication_word_frequency', 'TEXT_num_uncertainty_words_lexicon1',
]

# Normalize features
for col in x_columns:
    if col in df:
        df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

### Calculate 2-way Interaction Terms ###
# df['inter_2way_1'] = df['VOICE_pitch'] * df['TEXT_lexical_sophistication_word_frequency']
# x_columns.append('inter_2way_1')
# df['inter_2way_2'] = df['VOICE_articulation_rate'] * df['TEXT_sadness']
# x_columns.append('inter_2way_2')
# df['inter_2way_3'] = df['VOICE_articulation_rate'] * df['TEXT_knowledge']
# x_columns.append('inter_2way_3')
# df['inter_2way_4'] = df['VOICE_happy'] * df['TEXT_financial_sentiment_score']
# x_columns.append('inter_2way_4')
# df['inter_2way_5'] = df['VOICE_happy'] * df['TEXT_lexical_sophistication_word_frequency']
# x_columns.append('inter_2way_5')
# df['inter_2way_6'] = df['FACE_sad'] * df['TEXT_conflict']
# x_columns.append('inter_2way_6')

### Calculate 3-way Interaction Terms ###
# df['inter_1a']  = df['VOICE_pitch'] * df['FACE_happy']
# df['inter_1b']  = df['VOICE_pitch'] *  df['TEXT_knowledge']
# df['inter_1c']  = df['FACE_happy'] * df['TEXT_knowledge']
# df['inter_3way_1']  = df['VOICE_pitch'] * df['FACE_happy'] * df['TEXT_knowledge']
# x_columns.extend(['inter_1a', 'inter_1b', 'inter_1c', 'inter_3way_1'])

# df['inter_1a']  = df['VOICE_pitch'] * df['FACE_happy']
# df['inter_1b']  = df['VOICE_pitch'] * df['TEXT_lexical_diversity_mtld']
# df['inter_1c']  = df['FACE_happy'] * df['TEXT_lexical_diversity_mtld']
# df['inter_3way_2']  = df['VOICE_pitch'] * df['FACE_happy'] * df['TEXT_lexical_diversity_mtld']
# x_columns.extend(['inter_1a', 'inter_1b', 'inter_1c', 'inter_3way_2'])

# df['inter_1a']  = df['VOICE_pitch'] * df['FACE_happy']
# df['inter_1b']  = df['VOICE_pitch'] * df['TEXT_num_uncertainty_words_lexicon1']
# df['inter_1c']  = df['FACE_happy'] * df['TEXT_num_uncertainty_words_lexicon1']
# df['inter_3way_3']  = df['VOICE_pitch'] * df['FACE_happy'] * df['TEXT_num_uncertainty_words_lexicon1']
# x_columns.extend(['inter_1a', 'inter_1b', 'inter_1c', 'inter_3way_3'])

# df['inter_1a']  = df['VOICE_pitch'] * df['FACE_sad']
# df['inter_1b']  = df['VOICE_pitch'] * df['TEXT_conflict']
# df['inter_1c']  = df['FACE_sad'] * df['TEXT_conflict']
# df['inter_3way_4']  = df['VOICE_pitch'] * df['FACE_sad'] * df['TEXT_conflict']
# x_columns.extend(['inter_1a', 'inter_1b', 'inter_1c', 'inter_3way_4'])

# df['inter_1a']  = df['VOICE_pitch'] * df['FACE_sad']
# df['inter_1b']  = df['VOICE_pitch'] * df['TEXT_lexical_diversity_mtld']
# df['inter_1c']  = df['FACE_sad'] * df['TEXT_lexical_diversity_mtld']
# df['inter_3way_5']  = df['VOICE_pitch'] * df['FACE_sad'] * df['TEXT_lexical_diversity_mtld']
# x_columns.extend(['inter_1a', 'inter_1b', 'inter_1c', 'inter_3way_5'])

# df['inter_1a']  = df['VOICE_articulation_rate'] * df['FACE_smiling']
# df['inter_1b']  = df['VOICE_articulation_rate'] * df['TEXT_financial_sentiment_score']
# df['inter_1c']  = df['FACE_smiling'] * df['TEXT_financial_sentiment_score']
# df['inter_3way_6']  = df['VOICE_articulation_rate'] * df['FACE_smiling'] * df['TEXT_financial_sentiment_score']
# x_columns.extend(['inter_1a', 'inter_1b', 'inter_1c', 'inter_3way_6'])

# df['inter_1a']  = df['VOICE_articulation_rate'] * df['FACE_smiling']
# df['inter_1b']  = df['VOICE_articulation_rate'] * df['TEXT_lexical_sophistication_word_frequency']
# df['inter_1c']  = df['FACE_smiling'] * df['TEXT_lexical_sophistication_word_frequency']
# df['inter_3way_7']  = df['VOICE_articulation_rate'] * df['FACE_smiling'] * df['TEXT_lexical_sophistication_word_frequency']
# x_columns.extend(['inter_1a', 'inter_1b', 'inter_1c', 'inter_3way_7'])

# df['inter_1a']  = df['VOICE_articulation_rate'] * df['FACE_happy']
# df['inter_1b']  = df['VOICE_articulation_rate'] * df['TEXT_lexical_diversity_mtld']
# df['inter_1c']  = df['FACE_happy'] * df['TEXT_lexical_diversity_mtld']
# df['inter_3way_8']  = df['VOICE_articulation_rate'] * df['FACE_happy'] * df['TEXT_lexical_diversity_mtld']
# x_columns.extend(['inter_1a', 'inter_1b', 'inter_1c', 'inter_3way_8'])

# df['inter_1a']  = df['VOICE_articulation_rate'] * df['FACE_happy']
# df['inter_1b']  = df['VOICE_articulation_rate'] * df['TEXT_lexical_sophistication_word_frequency']
# df['inter_1c']  = df['FACE_happy'] * df['TEXT_lexical_sophistication_word_frequency']
# df['inter_3way_9']  = df['VOICE_articulation_rate'] * df['FACE_happy'] * df['TEXT_lexical_sophistication_word_frequency']
# x_columns.extend(['inter_1a', 'inter_1b', 'inter_1c', 'inter_3way_9'])

# df['inter_1a']  = df['VOICE_articulation_rate'] * df['FACE_neutral']
# df['inter_1b']  = df['VOICE_articulation_rate'] * df['TEXT_power']
# df['inter_1c']  = df['FACE_neutral'] * df['TEXT_power']
# df['inter_3way_10'] = df['VOICE_articulation_rate'] * df['FACE_neutral'] * df['TEXT_power']
# x_columns.extend(['inter_1a', 'inter_1b', 'inter_1c', 'inter_3way_10'])

# df['inter_1a']  = df['VOICE_articulation_rate'] * df['FACE_sad']
# df['inter_1b']  = df['VOICE_articulation_rate'] * df['TEXT_similarity']
# df['inter_1c']  = df['FACE_sad'] * df['TEXT_similarity']
# df['inter_3way_11'] = df['VOICE_articulation_rate'] * df['FACE_sad'] * df['TEXT_similarity']
# x_columns.extend(['inter_1a', 'inter_1b', 'inter_1c', 'inter_3way_11'])

# df['inter_1a']  = df['VOICE_articulation_rate'] * df['FACE_angry']
# df['inter_1b']  = df['VOICE_articulation_rate'] * df['TEXT_lexical_sophistication_word_frequency']
# df['inter_1c']  = df['FACE_angry'] * df['TEXT_lexical_sophistication_word_frequency']
# df['inter_3way_12'] = df['VOICE_articulation_rate'] * df['FACE_angry'] * df['TEXT_lexical_sophistication_word_frequency']
# x_columns.extend(['inter_1a', 'inter_1b', 'inter_1c', 'inter_3way_12'])

# df['inter_1a']  = df['VOICE_happy'] * df['FACE_smiling']
# df['inter_1b']  = df['VOICE_happy'] * df['TEXT_generic_sentiment_score']
# df['inter_1c']  = df['FACE_smiling'] * df['TEXT_generic_sentiment_score']
# df['inter_3way_13'] = df['VOICE_happy'] * df['FACE_smiling'] * df['TEXT_generic_sentiment_score']
# x_columns.extend(['inter_1a', 'inter_1b', 'inter_1c', 'inter_3way_13'])

# df['inter_1a']  = df['VOICE_happy'] * df['FACE_smiling']
# df['inter_1b']  = df['VOICE_happy'] * df['TEXT_lexical_sophistication_word_frequency']
# df['inter_1c']  = df['FACE_smiling'] * df['TEXT_lexical_sophistication_word_frequency']
# df['inter_3way_14'] = df['VOICE_happy'] * df['FACE_smiling'] * df['TEXT_lexical_sophistication_word_frequency']
# x_columns.extend(['inter_1a', 'inter_1b', 'inter_1c', 'inter_3way_14'])

# df['inter_1a']  = df['VOICE_happy'] * df['FACE_happy']
# df['inter_1b']  = df['VOICE_happy'] * df['TEXT_social_support']
# df['inter_1c']  = df['FACE_happy'] * df['TEXT_social_support']
# df['inter_3way_15'] = df['VOICE_happy'] * df['FACE_happy'] * df['TEXT_social_support']
# x_columns.extend(['inter_1a', 'inter_1b', 'inter_1c', 'inter_3way_15'])

# df['inter_1a']  = df['VOICE_happy'] * df['FACE_happy']
# df['inter_1b']  = df['VOICE_happy'] * df['TEXT_similarity']
# df['inter_1c']  = df['FACE_happy'] * df['TEXT_similarity']
# df['inter_3way_16'] = df['VOICE_happy'] * df['FACE_happy'] * df['TEXT_similarity']
# x_columns.extend(['inter_1a', 'inter_1b', 'inter_1c', 'inter_3way_16'])


### Calculate 3-way Interaction Terms ###
# df['inter_3way_1']  = df['VOICE_pitch'] * df['FACE_happy'] * df['TEXT_knowledge']
# x_columns.extend(['inter_3way_1'])

# df['inter_3way_2']  = df['VOICE_pitch'] * df['FACE_happy'] * df['TEXT_lexical_diversity_mtld']
# x_columns.extend(['inter_3way_2'])

# df['inter_3way_3']  = df['VOICE_pitch'] * df['FACE_happy'] * df['TEXT_num_uncertainty_words_lexicon1']
# x_columns.extend(['inter_3way_3'])

# df['inter_3way_4']  = df['VOICE_pitch'] * df['FACE_sad'] * df['TEXT_conflict']
# x_columns.extend(['inter_3way_4'])

# df['inter_3way_5']  = df['VOICE_pitch'] * df['FACE_sad'] * df['TEXT_lexical_diversity_mtld']
# x_columns.extend(['inter_3way_5'])

# df['inter_3way_6']  = df['VOICE_articulation_rate'] * df['FACE_smiling'] * df['TEXT_financial_sentiment_score']
# x_columns.extend(['inter_3way_6'])

# df['inter_3way_7']  = df['VOICE_articulation_rate'] * df['FACE_smiling'] * df['TEXT_lexical_sophistication_word_frequency']
# x_columns.extend(['inter_3way_7'])

# df['inter_3way_8']  = df['VOICE_articulation_rate'] * df['FACE_happy'] * df['TEXT_lexical_diversity_mtld']
# x_columns.extend(['inter_3way_8'])

# df['inter_3way_9']  = df['VOICE_articulation_rate'] * df['FACE_happy'] * df['TEXT_lexical_sophistication_word_frequency']
# x_columns.extend(['inter_3way_9'])

# df['inter_3way_10'] = df['VOICE_articulation_rate'] * df['FACE_neutral'] * df['TEXT_power']
# x_columns.extend(['inter_3way_10'])

# df['inter_3way_11'] = df['VOICE_articulation_rate'] * df['FACE_sad'] * df['TEXT_similarity']
# x_columns.extend(['inter_3way_11'])

# df['inter_3way_12'] = df['VOICE_articulation_rate'] * df['FACE_angry'] * df['TEXT_lexical_sophistication_word_frequency']
# x_columns.extend(['inter_3way_12'])

# df['inter_3way_13'] = df['VOICE_happy'] * df['FACE_smiling'] * df['TEXT_generic_sentiment_score']
# x_columns.extend(['inter_3way_13'])

# df['inter_3way_14'] = df['VOICE_happy'] * df['FACE_smiling'] * df['TEXT_lexical_sophistication_word_frequency']
# x_columns.extend(['inter_3way_14'])

# df['inter_3way_15'] = df['VOICE_happy'] * df['FACE_happy'] * df['TEXT_social_support']
# x_columns.extend(['inter_3way_15'])

df['inter_3way_16'] = df['VOICE_happy'] * df['FACE_happy'] * df['TEXT_similarity']
x_columns.extend(['inter_3way_16'])

# Group the data by graph id
grouped_df = df.groupby('Pitch_id')

# Create a list to store the graph data
graphs_males = []
graphs_females = []

# Iterate over the groups
for name, group in grouped_df:
    # Seperate males and females graphs & Create a PyTorch geometric data object
    gender_females = group['Gender==Female'].values[0]
    gender_males = group['Gender==Male'].values[0]
    
    # Extract the node features and labels
    x_pd = group[x_columns]
    x = x_pd.values    
#     print(x_pd.columns)
#     break

    # Extract the edges between nodes
    edges = group[['Segment_id']].T.values[0]
    edge_index = torch.tensor([edges[:-1], edges[1:]])
    
    # Extract the label for each graph
    y = group['DV_Received_Offer'].values[0]
    
    graph = Data(x=torch.tensor(x, dtype=torch.float), y=torch.tensor(y, dtype=torch.long), edge_index=edge_index)
    if gender_females == 1:
        graphs_females.append(graph)
    elif gender_males == 1:
        graphs_males.append(graph)
    
# print('Total graphs for females:', len(graphs_females))
# print('Total graphs for males:', len(graphs_males))
# # ### Store Graphs ###
# # torch.save(graphs_females, 'GNN_data_structure_females_v4_2wayInter.pt')
# # torch.save(graphs_males, 'GNN_data_structure_males_v4_2wayInter.pt')
# # ### Read Stored Graphs ###
# # graphs_females = torch.load('GNN_data_structure_females_v4_2wayInter.pt')
# # graphs_males = torch.load('GNN_data_structure_males_v4_2wayInter.pt')

num_features = graphs_females[0].x.shape[-1]

### Move to GPU ###
batch_females = Batch.from_data_list(graphs_females).to(device)
batch_males = Batch.from_data_list(graphs_males).to(device)

print('Num of features:', num_features)
print('Females:', len(graphs_females))
print('Males:', len(graphs_males))
print('Total:', len(graphs_females)+len(graphs_males))
print(len(batch_females.x))
print(len(batch_males.x))

######################################################
### Females - Train Model for Explainable AI ###
######################################################
print('Females - Train Model for Explainable AI...')
torch.manual_seed(11)
torch_geometric.seed_everything(11)
np.random.seed(11)

hidden_channels = 128
epochs = 150

batch_females = Batch.from_data_list(graphs_females).to(device)
model_females = train_model(num_features, hidden_channels, epochs, batch_females)

print('Finish training model for Females.')

######################################################
### Males - Train Model for Explainable AI ###
######################################################
print('Males - Train Model for Explainable AI...')
torch.manual_seed(11)
torch_geometric.seed_everything(11)
np.random.seed(11)

hidden_channels = 128
epochs = 240

batch_males = Batch.from_data_list(graphs_males).to(device)
model_males = train_model(num_features, hidden_channels, epochs, batch_males)

print('Finish training model for Males.')

######################################################
### Explainer Captum - Females ###
######################################################
print('Explanations Females...')
feat_import_entre_females = {}
feat_import_inves_females = {}
pos_i = 0
count_graphs = 0

for graph in graphs_females:
    num_nodes = graph.x.shape[0]
    num_feat  = graph.x.shape[1]
    batch = Batch.from_data_list([graph]).to(device)
    
    feat_import_entre_females[count_graphs] = [0]*num_feat
    feat_import_inves_females[count_graphs] = [0]*num_feat
    count_entre = 0
    count_inves = 0
    
    explainer_females = Explainer(
        model = model_females,
        algorithm = CaptumExplainer('IntegratedGradients'), # GNNExplainer(epochs=200, lr=0.02),  CaptumExplainer, PGExplainer, AttentionExplainer
        explanation_type = "phenomenon", # model phenomenon
        node_mask_type = "attributes", # object common_attributes attributes
        model_config=dict(
            mode        = "multiclass_classification",
            task_level  = "graph",
            return_type = "log_probs",
        )
    )
    explanation = explainer_females(
        x=batch.x,
        edge_index=batch.edge_index,
        target=batch.y,
        batch=batch.batch
    )
    explanations_list = explanation['node_mask']
    
    for i in range(0, num_nodes):
        node_features_list = graph.x[i].tolist() # batch.x[pos_i].tolist()
        node_importance_list = explanations_list[i].tolist()
        isEntre = int(node_features_list[0])
        
        if isEntre == 1:
            count_entre += 1
        else:
            count_inves += 1
        
        for j in range(0, num_feat):
            if isEntre == 1: # add feature importance to entrepreneurs
                feat_import_entre_females[count_graphs][j] += node_importance_list[j]
                
            else: # add feature importance to investors
                feat_import_inves_females[count_graphs][j] += node_importance_list[j]
    
    if count_inves == 0:
        count_inves = 1
    feat_import_entre_females[count_graphs] = [x*1.0/count_entre for x in feat_import_entre_females[count_graphs]]
    feat_import_inves_females[count_graphs] = [x*1.0/count_inves for x in feat_import_inves_females[count_graphs]]
    count_graphs += 1

######################################################
### Explainer Captum - Males ###
######################################################
print('Explanations Males...')
feat_import_entre_males = {}
feat_import_inves_males = {}
pos_i = 0
count_graphs = 0

for graph in graphs_males:
    num_nodes = graph.x.shape[0]
    num_feat  = graph.x.shape[1]
    batch = Batch.from_data_list([graph]).to(device)
    
    feat_import_entre_males[count_graphs] = [0]*num_feat
    feat_import_inves_males[count_graphs] = [0]*num_feat
    count_entre = 0
    count_inves = 0
    
    explainer_males = Explainer(
        model = model_males,
        algorithm = CaptumExplainer('IntegratedGradients'), # GNNExplainer(epochs=200, lr=0.02),  CaptumExplainer, PGExplainer, AttentionExplainer
        explanation_type = "phenomenon", # model phenomenon
        node_mask_type = "attributes", # object common_attributes attributes
        model_config=dict(
            mode        = "multiclass_classification",
            task_level  = "graph",
            return_type = "log_probs",
        )
    )
    explanation = explainer_males(
        x=batch.x,
        edge_index=batch.edge_index,
        target=batch.y,
        batch=batch.batch
    )
    explanations_list = explanation['node_mask']
    
    for i in range(0, num_nodes):
        node_features_list = graph.x[i].tolist() # batch.x[pos_i].tolist()
        node_importance_list = explanations_list[i].tolist()
        isEntre = int(node_features_list[0])
        
        if isEntre == 1:
            count_entre += 1
        else:
            count_inves += 1
        
        for j in range(0, num_feat):
            if isEntre == 1: # add feature importance to entrepreneurs
                feat_import_entre_males[count_graphs][j] += node_importance_list[j]
                
            else: # add feature importance to investors
                feat_import_inves_males[count_graphs][j] += node_importance_list[j]
    
    if count_inves == 0:
        count_inves = 1
    feat_import_entre_males[count_graphs] = [x*1.0/count_entre for x in feat_import_entre_males[count_graphs]]
    feat_import_inves_males[count_graphs] = [x*1.0/count_inves for x in feat_import_inves_males[count_graphs]]
    count_graphs += 1

print('Finish CAPTUM.')

######################################################
### Export Importance of Features ###
######################################################
for i in range(0, len(graphs_females)):
    last_item = 0
    for elem in feat_import_entre_females[i]:
        last_item = str(elem)
    print(last_item)
for i in range(0, len(graphs_males)):
    last_item = 0
    for elem in feat_import_entre_males[i]:
        last_item = str(elem)
    print(last_item)

Num of features: 55
Females: 272
Males: 625
Total: 897
15617
32873
Females - Train Model for Explainable AI...
Finish training model for Females.
Males - Train Model for Explainable AI...
Finish training model for Males.
Explanations Females...
Explanations Males...
Finish CAPTUM.
0.0011335219566611283
5.092406211037956e-10
-1.34576740555478e-08
2.3114518028138525e-08
3.240586052366987e-05
-9.739542648701068e-07
8.510157036602524e-08
-0.00029629402103880514
-4.849941309369136e-07
-1.2134708501712545e-10
3.3311856567149174e-05
-7.652904136217894e-06
3.4809089711530958e-12
9.754191845781152e-07
-1.176823583468443e-05
-0.00017334259400377528
0.00023038486479099975
2.5471755425702028e-05
6.417323518095035e-08
0.0
4.19439800486183e-06
3.473508111734468e-08
0.0
-2.6486286321586163e-06
1.1155886431861712e-06
4.260704736946701e-06
0.0
-4.889449467078517e-08
5.334456557315504e-08
-6.1421171739795135e-06
0.00027376928794986224
0.0
9.978734490492005e-11
-4.494074014222376e-05
-2.1872089441555216e